# Retirement Simulation: Life Cycle Investing Strategies

## FINC 450: Life Cycle Investing

This notebook implements a Monte Carlo simulation comparing retirement spending strategies. Our goals are to understand:

1. How **duration matching** protects funded status against interest rate risk
2. How **variable consumption** reduces default probability
3. The trade-offs between these approaches

---

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import our simulation modules
from retirement_simulation import (
    EconomicParams,
    BondParams,
    SimulationParams,
    run_monte_carlo,
    compute_summary_stats,
    STRATEGIES,
    liability_pv,
    liability_duration,
)
from visualizations import (
    plot_duration_matching_intuition,
    plot_strategy_comparison_bars,
    plot_wealth_paths_spaghetti,
    plot_consumption_paths,
    plot_final_wealth_distribution,
    plot_interest_rate_scenarios,
    create_summary_table,
)

%matplotlib inline
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 150

## 1. The Setup: A Retiree's Problem

Consider a retiree with:
- **Initial wealth**: $2.5 million
- **Annual spending need**: $100,000 (real)
- **Retirement horizon**: 30 years
- **Stock allocation**: 40% (fixed)

**Key question**: How should the retiree allocate the remaining 60% in bonds, and how should they adjust spending over time?

In [ ]:
# Simulation parameters
sim_params = SimulationParams(
    initial_wealth=2_500_000,
    annual_consumption=100_000,
    horizon=30,
    stock_weight=0.40,
    n_simulations=10_000,
    random_seed=42
)

# Economic environment parameters
econ_params = EconomicParams(
    r_bar=0.03,        # 3% long-run real rate
    phi=0.85,          # Interest rate persistence
    sigma_r=0.012,     # Rate volatility
    mu_excess=0.04,    # 4% equity risk premium
    sigma_s=0.18,      # Stock volatility (18%)
    rho=-0.2,          # Negative correlation: stocks fall when rates rise
    r_floor=0.001      # 0.1% rate floor
)

# Bond parameters
bond_params = BondParams(
    D_mm=0.25,         # Money market duration
    D_lb=15.0          # Long bond duration
)

print("Simulation Configuration:")
print(f"  Initial wealth: ${sim_params.initial_wealth:,.0f}")
print(f"  Annual consumption target: ${sim_params.annual_consumption:,.0f}")
print(f"  Horizon: {sim_params.horizon} years")
print(f"  Stock allocation: {sim_params.stock_weight*100:.0f}%")
print(f"  Number of simulations: {sim_params.n_simulations:,}")

## 2. Understanding Liabilities

The retiree's **liability** is the present value of future consumption needs:

$$PV_{liab}(r) = C \times \frac{1 - (1+r)^{-T}}{r}$$

The **modified duration** of liabilities measures sensitivity to rate changes:

$$D_{liab}(r, T) = \frac{1}{PV_{liab}} \times \sum_{t=1}^{T} \frac{t \times C}{(1+r)^{t+1}}$$

In [ ]:
# Calculate liability PV and duration at different rates
rates = [0.01, 0.02, 0.03, 0.04, 0.05]
C = sim_params.annual_consumption
T = sim_params.horizon

print("Liability Analysis at Different Interest Rates:")
print("=" * 60)
print(f"{'Rate':>8} {'PV Liabilities':>18} {'Duration':>12}")
print("-" * 60)

for r in rates:
    pv = liability_pv(C, r, T)
    dur = liability_duration(C, r, T)
    print(f"{r*100:>7.1f}% {pv:>17,.0f} {dur:>12.1f} yrs")

print("\nKey Insight: Lower rates mean HIGHER liability PV and LONGER duration.")
print("This is why low-rate environments are particularly challenging for retirees.")

## 3. The Four Strategies

We compare four retirement strategies that combine:

**Bond Allocation:**
- **Money Market (MM)**: All bonds in short-duration money market (D ≈ 0.25)
- **Duration Matching (DM)**: Match bond duration to liability duration

**Consumption Rule:**
- **Fixed**: Spend $100k/year regardless of wealth
- **Variable**: Spend a percentage of remaining wealth

| Strategy | Bond Duration | Consumption |
|----------|--------------|-------------|
| MM + Fixed | D ≈ 0 | Fixed $100k/yr |
| DurMatch + Fixed | D = D_liab | Fixed $100k/yr |
| MM + Variable | D ≈ 0 | % of wealth |
| DurMatch + Variable | D = D_liab | % of wealth |

In [ ]:
# Print strategy details
print("Strategies to Compare:")
print("=" * 50)
for i, strategy in enumerate(STRATEGIES, 1):
    print(f"{i}. {strategy}")

## 4. Run the Monte Carlo Simulation

We now simulate 10,000 retirement scenarios for each strategy.

In [ ]:
# Run the simulation
print("Running Monte Carlo simulation...")
results, rates, stock_returns = run_monte_carlo(
    econ_params=econ_params,
    bond_params=bond_params,
    sim_params=sim_params
)
print(f"Completed {sim_params.n_simulations:,} simulations for {len(STRATEGIES)} strategies.")

## 5. Results Summary

In [ ]:
# Display summary statistics
summary_df = compute_summary_stats(results)
print("\nStrategy Comparison Summary:")
print("=" * 80)
display(summary_df.round(2))

In [ ]:
# Visual comparison of key metrics
fig = plot_strategy_comparison_bars(results)
plt.suptitle('Strategy Performance Comparison', fontsize=14, y=1.02)
plt.show()

### Key Observations:

1. **Variable consumption eliminates default risk** - By definition, you can't run out of money if you spend a percentage of what you have.

2. **Fixed consumption has material default risk** - Even with $2.5M starting wealth, bad sequences of returns can lead to ruin.

3. **Duration matching has mixed effects** - It stabilizes funded status but introduces more portfolio volatility.

## 6. Understanding Duration Matching

Duration matching is an **immunization strategy** that aligns asset duration with liability duration.

**Why it works:**
- When rates fall: Bond prices rise, but so does the PV of liabilities
- When rates rise: Bond prices fall, but liability PV also falls

The **funded ratio** (Assets/Liabilities) stays stable.

In [ ]:
# Find an interesting scenario where rates move significantly
rate_changes = rates[:, -1] - rates[:, 0]  # Total rate change over 30 years

# Find a scenario with substantial rate increase
rising_rate_idx = np.argmax(rate_changes)
print(f"Sample path with rising rates:")
print(f"  Initial rate: {rates[rising_rate_idx, 0]*100:.2f}%")
print(f"  Final rate: {rates[rising_rate_idx, -1]*100:.2f}%")
print(f"  Rate change: {rate_changes[rising_rate_idx]*100:+.2f}%")

In [ ]:
# Duration matching intuition visualization
fig = plot_duration_matching_intuition(
    rates=rates,
    wealth_paths_mm=results['MM + Fixed'].wealth_paths,
    wealth_paths_dm=results['DurMatch + Fixed'].wealth_paths,
    consumption_target=sim_params.annual_consumption,
    horizon=sim_params.horizon,
    sample_idx=rising_rate_idx
)
plt.suptitle('Duration Matching Intuition: A Rising Rate Scenario', fontsize=14, y=1.02)
plt.show()

### Interpretation:

- **Top left**: Interest rate path over the 30-year horizon
- **Top right**: Money market strategy - assets and liabilities diverge when rates change
- **Bottom left**: Duration-matched strategy - assets track liabilities more closely
- **Bottom right**: Funded ratio comparison - duration matching provides more stable funding

**Caveat**: Duration matching introduces MORE portfolio volatility. It's a trade-off between funded-status stability and asset volatility.

## 7. Wealth Path Analysis

In [ ]:
# Spaghetti plot of wealth paths
fig = plot_wealth_paths_spaghetti(results, n_paths=100)
plt.suptitle('Wealth Trajectories by Strategy (100 Sample Paths)', fontsize=14, y=1.02)
plt.show()

### Observations:

- **Red paths** indicate scenarios that defaulted (hit zero)
- Variable consumption strategies never hit zero (by construction)
- Duration-matched strategies show wider dispersion in wealth outcomes
- The shaded region shows the 10th-90th percentile band

## 8. Consumption Path Analysis

In [ ]:
# Consumption paths
fig = plot_consumption_paths(results, sim_params.annual_consumption, n_paths=50)
plt.suptitle('Consumption Paths by Strategy', fontsize=14, y=1.02)
plt.show()

### Observations:

- **Fixed consumption** strategies maintain the target until default (then consumption drops to zero)
- **Variable consumption** strategies show more year-to-year volatility but never hit zero
- The trade-off: consumption certainty vs. default risk

## 9. Final Wealth Distribution

In [ ]:
# Final wealth distribution
fig = plot_final_wealth_distribution(results)
plt.show()

## 10. Interest Rate Scenarios

In [ ]:
# Visualize the interest rate scenarios
fig = plot_interest_rate_scenarios(rates, n_paths=100)
plt.show()

## 11. Key Teaching Points

### 1. Duration Matching Protects Funded Status

Duration matching ensures that when interest rates change, assets and liabilities move together. This protects the **funded status** (Assets/Liabilities ratio).

However, it introduces more **portfolio volatility** because long-duration bonds are more volatile than money market instruments.

### 2. Variable Consumption Eliminates Default Risk

By spending a percentage of wealth rather than a fixed amount, the retiree can never run out of money (mathematically impossible).

The cost is **consumption uncertainty** - spending varies year to year.

### 3. The Two Hedges Address Different Risks

| Strategy | Risk Hedged | Cost |
|----------|------------|------|
| Duration Matching | Interest rate risk | Higher portfolio volatility |
| Variable Consumption | Sequence-of-returns risk | Consumption uncertainty |

### 4. Low-Rate Environment Considerations

When rates are already low (near the floor):
- Rates are more likely to rise than fall further
- Duration matching may hurt because long bonds will lose value if rates rise
- The asymmetry of rate movements matters

## 12. Sensitivity Analysis: Starting Rate Environment

In [ ]:
# How do results change with different starting rates?
print("Running sensitivity analysis for different starting interest rates...")
print("(This may take a minute)")

starting_rates = [0.01, 0.03, 0.05]
sensitivity_results = {}

for r0 in starting_rates:
    results_r0, _, _ = run_monte_carlo(
        econ_params=econ_params,
        bond_params=bond_params,
        sim_params=SimulationParams(
            initial_wealth=sim_params.initial_wealth,
            annual_consumption=sim_params.annual_consumption,
            horizon=sim_params.horizon,
            stock_weight=sim_params.stock_weight,
            n_simulations=5000,  # Fewer sims for speed
            random_seed=42
        ),
        initial_rate=r0
    )
    sensitivity_results[r0] = results_r0
    print(f"  Completed r0 = {r0*100:.0f}%")

print("Done!")

In [ ]:
# Compare default rates across starting rate environments
print("\nDefault Rates by Starting Interest Rate:")
print("=" * 70)
print(f"{'Strategy':<25} {'r0=1%':>12} {'r0=3%':>12} {'r0=5%':>12}")
print("-" * 70)

for strategy_name in [str(s) for s in STRATEGIES]:
    rates_str = ""
    for r0 in starting_rates:
        default_rate = 100 * sensitivity_results[r0][strategy_name].defaulted.mean()
        rates_str += f"{default_rate:>12.1f}%"
    print(f"{strategy_name:<25}{rates_str}")

### Interpretation:

- **Lower starting rates** generally lead to higher default probabilities (lower expected returns)
- **Duration matching** may perform worse in low-rate environments if rates are likely to rise
- **Variable consumption** remains robust across all rate environments (no default by construction)

## 13. Summary and Discussion Questions

### Summary Table

In [ ]:
# Final summary
print(create_summary_table(results))

### Discussion Questions for Class

1. **Risk trade-offs**: A retiree can choose between default risk (fixed consumption) and consumption uncertainty (variable consumption). Which would you prefer? Does your answer depend on other sources of income (Social Security, pensions)?

2. **Duration matching in practice**: Given that liability duration decreases as the retiree ages, how frequently should the portfolio be rebalanced? What are the transaction costs?

3. **Low-rate environments**: How should the analysis change if we believe rates are more likely to rise than fall from current levels? Does duration matching become less attractive?

4. **Bequest motives**: How would the optimal strategy change if the retiree cares about leaving wealth to heirs?

5. **Human capital and flexibility**: Young retirees might have the option to return to work if wealth falls. How does this "human capital option" affect optimal strategy choice?

6. **Beyond the model**: What important real-world considerations are missing from this simulation? (Hint: inflation uncertainty, health shocks, housing, Social Security timing, etc.)

---

## Appendix: Model Details

### Economic Environment (VAR Structure)

**Interest Rates** follow an AR(1) process:
$$r_{t+1} = \bar{r} + \phi(r_t - \bar{r}) + \varepsilon_{r,t}$$

**Stock Returns** equal the risk-free rate plus an IID excess return:
$$R_{stock,t} = r_t + \mu_{excess} + \varepsilon_{s,t}$$

**Correlation**: Rate shocks and stock shocks have correlation $\rho = -0.2$

### Bond Returns

$$R_{bond,t} = r_t - D \times \Delta r_{t+1}$$

### Parameters Used

| Parameter | Value | Description |
|-----------|-------|-------------|
| $\bar{r}$ | 3% | Long-run mean real rate |
| $\phi$ | 0.85 | Rate persistence |
| $\sigma_r$ | 1.2% | Rate shock volatility |
| $\mu_{excess}$ | 4% | Equity risk premium |
| $\sigma_s$ | 18% | Stock return volatility |
| $\rho$ | -0.2 | Shock correlation |
| $D_{mm}$ | 0.25 | Money market duration |
| $D_{lb}$ | 15 | Long bond duration |